# <font color='red'>Random Forest Algorithm </font>
&nbsp; <a href = "https://scholar.google.co.in/citations?user=pTdyt0YAAAAJ&hl=en"> <font color='blue'> By Dr. Mohendra Roy (Ph. D.)</font> </a>

&nbsp;<font color='blue'>Design Engineer at A. I. Medicines</font>&nbsp; &copy;  http://aimedicines.com/

**Here we will impliment as well as explain the steps of Random Forest algorithm.** 

Random Forest algorithm is constituents of dicission trees. The voted output(the output the majority of trees provide) for a perticular test row is selected from the collection of trees from the forest. There fore in this algorithm we have to make the following functions:

1) Random Forest consisting of decision trees

2) Functions for dicision tree


** The Random Forest function **

In [34]:
def Random_Forest(training_data, test_data, maximum_branch, minimum_sample, subsample_ratio, tree_number, no_of_features):
	trees = list()
	for i in range(tree_number):
		sub_sample = Make_Subsample(training_data, subsample_ratio) # "Make_Subsample" is a user defined function 
		tree = Make_Tree(sub_sample, maximum_branch, minimum_sample, no_of_features)#"Make_Tree" is a user defined function
		trees.append(tree)
	output = [Voted_Output(trees, row) for row in test_data]
	return(output)

Here **"training_data"** argument is the dataset for training the **<font color='red'>"Random_Forest"</font>** function. Subsiquently **"test_data"** is the dataset to test the performance of the algorithm. 

**maximum_branch** is the agrument for the maximum _allowed branch_ or _depth_ in a tree. Similarly ** minimum_sample ** is the minimum number of sample required in a **_node_** for further brancing from that node. 


**subsample_ratio** is the ratio at which the training sample will be randomly devided to subsamples for the training of the trees.

**tree_number** is the total number of trees that preset in the random forest

**no_of_features** is the total number of random features that will use by a tree for the training. Each tree will use different sets of features for training. and the minimum number of features required for the traing of a tree is given by ---  

$$ Number \ of \ Features = \sqrt{Length \ of\ a\ row\ in\  dataset } $$

**<font color = 'red'> Note: </font>** Here we use the following user defined functions :


** <font color = 'blue'> 1) Make_Subsample,    2)  Make_Tree,  3) Voted_Output </font> **

Further we will explain these functions.


In [35]:
# functin for making subsample from the training sample. 

from random import randrange # "randrange" generates random number for a given range

def Make_Subsample(dataset, subsample_ratio):
	sample = list()
	n_sample = round(len(dataset) * subsample_ratio)
	while len(sample) < n_sample:
		index = randrange(len(dataset))
		sample.append(dataset[index])
	return sample


The above function will return a random subset of the given dataset. 

In [36]:
# Function for making tree

def Make_Tree(training_data, maximum_branch, minimum_sample, no_of_features):
	root = Make_Label(training_data, no_of_features) # "Make_Label" is a user defined function 
	Child_Split(root, maximum_branch, minimum_sample, no_of_features, 1) # Child_Split is a user defined function
	return root

The above function will make tree. In this function we used the <font color = blue> **Make_Label** </font> and <font color = blue> **Child_Split**</font> functions. We will discuss about these after the **<font color = blue> Voted_Output</font>** function. 

In [37]:
# Function for calculating voted output 
def Voted_Output(trees, test_row):
	p = [Predict(tree, test_row) for tree in trees] # "Predict" is a user defined function
	return max(set(p), key=p.count) # This will return the maximum occured number in the row in **"p"** provided by the
# Predict function

In the above function, the **"trees"** argument is retured by the  **<font color = 'blue'> Make_Tree </font> ** function. Again in the above function we used the **<font color = 'blue'> Predict </font> ** function. The detail of the function is as given bellow. 

In [38]:
# Function to predict the test row to be in a perticular group

def Predict(node, test_row):
	if test_row[node['index']] < node['value']:
		if isinstance(node['left'], dict): 
			return Predict(node['left'], test_row)
		else:
			return node['left']
	else:
		if isinstance(node['right'], dict):
			return Predict(node['right'], test_row)
		else:
			return node['right']

Here the argument **"node"** is returned by the **<font color = 'blue'> Make_Tree </font>** function. Which contain the dictionary containing the **"index"**, **"value"**, **label**( or label_value) of the termination node and if applicable, sub groups of all of these for sub branch.

Note: ***"isinstance"*** function  return, whether an object is a instance of a class or of a subclass thereof. Here if the left(or right) itsef contain the dictionaly(**dict**) then it will again operate the "Predict" function on the left(or right)
itself.

In [39]:
# Function to make label

def Make_Label(dataset, no_of_features):
	class_labels = list(set(row[-1] for row in dataset))
	bindex, bvalue, bscore, bgroups = 1000, 1000, 1000, None # Innitial labels(with values) of the dictionary 
	features = list()
	while len(features) < no_of_features:
		index = randrange(len(dataset[0])-1)
		if index not in features:
			features.append(index)
	for index in features:
		for row in dataset:
			groups = Data_Split(index, row[index], dataset) # "Data_Split" is a user defined function
			gini = Gini_Index(groups, class_labels) # "Gini Index" is a user defined function 
			if gini < bscore:
				bindex, bvalue, bscore, bgroups = index, row[index], gini, groups
	return {'index':bindex, 'value':bvalue, 'groups':bgroups}

Here the ***"class_labels"*** contains the labels or the label numbers, each from a perticular Row of the dataset(train set). 

Here we used the **<font color = 'blue'> Data_Split</font>** and **<font color = 'blue'> Gini_Index </font>** build in functions. These are as explained in bellow. 

In [40]:
# Function to split the data based on the index and value that have supplied to it.

def Data_Split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right

In [41]:
# function for making the sub nodes by spliting the parent node to obtain branches of the tree and eventually end node.

def Child_Split(node,maximum_branch, minimum_sample, no_of_features, current_depth):
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = Terminate(left + right) # "Terminate" is a user defined function
		return
	# check for max depth
	if current_depth >= maximum_branch:
		node['left'], node['right'] = Terminate(left), Terminate(right)
		return
	# process left child
	if len(left) <= minimum_sample:
		node['left'] = Terminate(left)
	else:
		node['left'] = Make_Label(left, n_features)
		Child_Split(node['left'],maximum_branch, minimum_sample, no_of_features, current_depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = Terminate(right)
	else:
		node['right'] = Make_Label(right, n_features)
		Child_Split(node['right'], maximum_branch, minimum_sample, no_of_features, current_depth+1)

Here we used a user defined functin called **<font color = blue> Terminate </font>**. Bellow we will frame this function. 

In [42]:
def Terminate(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)

The ***"return max(set(outcomes), key=outcomes.count)"*** command gives the output of the of the maximum occured number in the **outcomes** array.

We know that Gini Index is one of the cost function that is used in the decision tree algorithm. And this is given by:$$ Gini(f) = \sum_{i=1}^{J} c_i (1-c_i) $$ Where c is the appearance of the class in a perticular group and i = 1,...J are class types ( or class labels). This is for a perticular one group. For actual Gini Index we have to take the weighted sum. $$ Gini\ Index(f) = \sum_{g=1}^{n}  \sum_{i=1}^{J} \frac{1}{Length(G_n)} c_i (1-c_i) . $$ Where G is the group and g =1,..n is the number of groups. 


We can impliment this as a function as described in below.

In [43]:
# function to determine the Gini Index
def Gini_Index(groups, class_labels):
	gini = 0.0
	for class_value in class_labels:
		for group in groups:
			size = len(group)
			if size == 0:
				continue # This is to avoid the empty groups
			proportion = [row[-1] for row in group].count(class_value) / float(size)
			gini += (proportion * (1.0 - proportion))
	return gini

Till now we have build the Random forest algorithm. Now we will write function to evaluate the performance of the Random forest algorithm. 


In [2]:
# Function to evaluate the Random Forest algorithm 

def Evaluate_Algorithm(dataset, algorithm, number_of_folds, *args): # *args contain list of arguments
	folds = Make_Fold(dataset, number_of_folds) # "Make_Fold" is a user define function
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold) # this is because we will use the current fold as a test set
		train_set = sum(train_set, []) # this will make the train set containing purely one group( only one matrix)
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None # to remove the label from the test set
		predicted = algorithm(train_set, test_set, *args) # Further we will replace the "algorithm" with "Random_Forest"
		actual = [row[-1] for row in fold] # copy the labels of the test set
		accuracy = Accuracy(actual, predicted) # "Accuracy" is the user defined function
		scores.append(accuracy)
	return scores

Here we used the ** <font color=blue> Make_Fold </font>** and ** <font color=blue> Accuracy </font>** user defined function

In [3]:
#  function to Make Fold of the database to get the training and testing sub dataset 

def Make_Fold(dataset, number_of_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / number_of_folds)
	for i in range(number_of_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

In [4]:
# function to find the accuracy betwwen the predicted and actual data

def Accuracy(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

Now we will make functions to load the file and make ready the dataset for apply in Random Forest algorithm

In [ ]:
from csv import reader
from math import sqrt

# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

.




** N.B.** Here I intetinaly write the user defined functions starting with capital letter to differentiate between the build in functions and user defined functions. 




.

.



**Acknowledgement:** I deeply thankful to <a href = "http://machinelearningmastery.com/about/"> <font color='blue'> Jason Brownlee </font> </a>


.

**References:** 

[1] http://machinelearningmastery.com/blog/

[2] Machine Learning: A Probabilistic Perspective. A textbook by Kevin R Murphy. ISBN: 978-0-262-01802-9

[3] 